# script to annotat GWAS files with the allelic imbalance files

In [16]:
import pandas as pd
import numpy as np
import os
import scipy
import pybedtools as pbed 
import subprocess as sub
from multiprocessing import Pool
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.multitest
import gzip
import subprocess
import vcf
import io
import functools
import requests
import json
import sys
import pickle

base_dir = "http://bartzabel.ls.manchester.ac.uk/orozcolab/SNP2Mechanism/"
plt.rcParams['svg.fonttype'] = 'none'

In [17]:
all_SNPs_all = pd.read_csv("../ATAC_allelic_imbalance/.local/results/ATAC_ALL_allelic_imbalance_with_betabinom.csv.gz", index_col = 0)

In [18]:
def convert_set(el):
    if isinstance(el, list) or isinstance(el, set):
        if len(el) > 1:
            return ", ".join(str(e) for e in el)
        else:
            return list(el)[0]
    else:
        return el

In [19]:
def annotate(file, name):
    snps_df  = pd.read_csv(file, sep = "\t", header = None)
    snps_df.columns = "chr start end name score".split()
    snps_df["loci"] = snps_df["name"].str.split("_").str[-1]
    snps_df["snp"] = snps_df["name"].str.split("_").str[0]
    annotated_snps_df_all = snps_df.merge(all_SNPs_all[["ID","corrected_p_val_greater","corrected_p_val_less","tot_REF","tot_ALT","ratio","n_pat","TF_remap","TF_JASPAR","ATAC_hic_corr_score", "eQTLgen_gene", "eQTLgen_symbol", "eQTLgen_pval","hsc_genes", "tcell_genes", "all_genes","svalues_betabinom"]], 
                                          left_on = "snp", right_on = "ID", how = "left")
    annotated_snps_df_all = annotated_snps_df_all.applymap(convert_set)
    annotated_snps_df_all.to_csv(f"results_allele_specific_snp/{name}_ALL.csv")


In [20]:
annotate(f"{base_dir}/other_tracks/GWAS/tsoi2017_LD_0.8_hg38.bed", "psoriasis_tsoi2017")
annotate(f"{base_dir}/other_tracks/GWAS/RAmetagwas_all_hg38.ld.bed", "RAmeta")
annotate(f"{base_dir}/other_tracks/GWAS/PsA_vs_controls_metagwas_significant.ld.hg38.bed", "PsA_meta")
annotate(f"{base_dir}/other_tracks/GWAS/suggestive_snps_hg38_ld.bed", "JIA_suggestive")
annotate(f"{base_dir}/other_tracks/GWAS/JIA_credible_snps_hg38.bed", "JIA_credible")
annotate(f"{base_dir}/other_tracks/GWAS/elena_hg38.ld.bed", "SSc_elena")

/tmp/ipykernel_81782/3912690093.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annotated_snps_df_all = annotated_snps_df_all.applymap(convert_set)
/tmp/ipykernel_81782/3912690093.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annotated_snps_df_all = annotated_snps_df_all.applymap(convert_set)
/tmp/ipykernel_81782/3912690093.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annotated_snps_df_all = annotated_snps_df_all.applymap(convert_set)
/tmp/ipykernel_81782/3912690093.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annotated_snps_df_all = annotated_snps_df_all.applymap(convert_set)
/tmp/ipykernel_81782/3912690093.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annotated_snps_df_all = annotated_snps_df_all.applymap(convert_set)
/tmp/ipykernel_81782/3912690093.py:8: FutureWarnin

## alternate script that also adds most severe consequence from ensembl

In [20]:
def get_most_severe_consequences(variants, species='human', max_batch_size=200):
    """
    Function to get the most severe consequence for each variant ID using the Ensembl API.
    
    :param variants: A list of variant IDs.
    :param species: The species to query (default: 'human').
    :param max_batch_size: The maximum number of elements per request (default: 200).
    :return: A pandas DataFrame with the variant ID and the most severe consequence.
    """
    def process_batch(batch):
        data = json.dumps({"ids": batch})
        response = requests.post(server + ext, headers=headers, data=data)

        if not response.ok:
            response.raise_for_status()
            sys.exit()

        return response.json()

    server = "https://rest.ensembl.org"
    ext = f"/vep/{species}/id"
    headers = {"Content-Type": "application/json", "Accept": "application/json"}

    results = []

    for i in range(0, len(variants), max_batch_size):
        batch = variants[i:i + max_batch_size]
        vep_data = process_batch(batch)

        for item in vep_data:
            if 'id' in item and 'most_severe_consequence' in item:
                results.append({
                    'snp': item['id'],
                    'most_severe_consequence': item['most_severe_consequence']
                })

    df = pd.DataFrame(results)
    return df


In [21]:
def annotate(file, name):
    snps_df  = pd.read_csv(file, sep = "\t", header = None)
    snps_df.columns = "chr start end name score".split()
    snps_df["loci"] = snps_df["name"].str.split("_").str[-1]
    snps_df["snp"] = snps_df["name"].str.split("_").str[0]
    annotated_snps_df_all = snps_df.merge(all_SNPs_all[["ID","corrected_p_val_greater","corrected_p_val_less","tot_REF","tot_ALT","ratio","n_pat","TF_remap","TF_JASPAR","ATAC_hic_corr_score", "eQTLgen_gene", "eQTLgen_symbol", "eQTLgen_pval","CD4_lowest_allele_specific","CD8_lowest_allele_specific","hsc_genes", "tcell_genes", "all_genes"]], 
                                          left_on = "snp", right_on = "ID", how = "left")
    ensembl_effect = get_most_severe_consequences(snps_df["snp"].to_list())
    ensembl_effect = ensembl_effect.drop_duplicates(subset = "snp")
    annotated_snps_df_all = annotated_snps_df_all.merge(ensembl_effect, on = "snp", how = "left")
    annotated_snps_df_all = annotated_snps_df_all.applymap(convert_set)
    annotated_snps_df_all.to_csv(f"{base_dir}/ATAC_allelic_imbalance/results_allele_specific_snp/{name}_ALL_with_consequence.csv")

In [22]:
annotate(f"{base_dir}/metadata/GWAS_files/tsoi2017_LD_0.8_hg38.bed", "psoriasis_tsoi2017")
annotate(f"{base_dir}/metadata/GWAS_files/RAmetagwas_all_hg38.ld.bed", "RAmeta")
annotate(f"{base_dir}/metadata/GWAS_files/PsA_vs_controls_metagwas_significant.ld.hg38.bed", "PsA_meta")
annotate(f"{base_dir}/metadata/GWAS_files/suggestive_snps_hg38_ld.bed", "JIA_suggestive")
annotate(f"{base_dir}/metadata/GWAS_files/JIA_credible_snps_hg38.bed", "JIA_credible")
annotate(f"{base_dir}/metadata/GWAS_files/elena_hg38.ld.bed", "SSc_elena")